<a href="https://colab.research.google.com/github/AqueeqAzam/reinforcement-learning-for-robotic/blob/main/reinforcement_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# `Fundamental Concept`

Agent: The actor performing whthin the env.

Env: The region in which the actor can perform in.

Action: The actor can take action within the env.

Reward + Observation: Actor recevied a reward and a view of what the env look like after performing on it.

Agent --> Env --> Action --> Reward + obs

# `Setup dependencies`

In [ ]:
!pip install stable-baselines3[extra]

In [3]:
import os
# It provides functions for interacting with the operating system, such as creating directories, listing files and directories,pen_spark

import gym
# It  is a popular toolkit for developing and testing reinforcement learning (RL) environments.
# It provides a standardized interface for creating and interacting with RL tasks.

from stable_baselines3 import PPO
# Importing the PPO class allows you to create an instance of a PPO agent to train in an RL environment.

from stable_baselines3.common.evaluation import evaluate_policy
# This function helps you evaluate the performance of an RL agent on a given environment.pen_spark

from stable_baselines3.common.vec_env import DummyVecEnv
# It can be beneficial for certain RL algorithms or training setups.

# `Load Env`

In [25]:
env_name = 'CartPole-v1'

#  the gym.make() function, which creates an instance of the environment you specified in env_name
env = gym.make(env_name)

# display env name
env_name

# The reset() method returns the initial observation (state) of the environment, which the agent can use to make decisions.
env.reset()

# This method attempts to visually display the environment, which can be helpful for debugging or understanding the state of the environment.
env.render()

# The action_space defines the set of possible actions the agent can take (e.g., left force, right force),
env.action_space.sample()

# the observation_space defines the range of possible observations (states) the agent can receive from the environment (e.g., cart position, pole angle).
env.observation_space.sample()

# This line calls the close() method on the environment.
env.close()

env.step(1)


(array([-0.03690413,  0.20433587,  0.01245518, -0.2731477 ], dtype=float32),
 1.0,
 False,
 {})

# `Setup Env`

In [ ]:
episodes = 5
for episode in range(1, episodes+1):
  print(episode)
  state = env.reset()
  print(state)

  print(env.action_space.sample())
  print(env.observation_space.sample())

In [ ]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        # env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

In [ ]:
env.close()

# `Understanding Env`

In [37]:
env.action_space

env.action_space.sample()

env.observation_space
env.observation_space.sample()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([-2.8230591e+00, -1.6741126e+38,  3.3211878e-01, -3.2379831e+38],
      dtype=float32)

Training

# `Simple Project`

In [ ]:
import gym
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Hyperparameters
learning_rate = 0.01
discount_factor = 0.95
episodes = 1000

# Create the environment
env = gym.make('CartPole-v1')

# Define the agent's policy network (model)
model = Sequential([
  Dense(16, activation='relu', input_shape=env.observation_space.shape),
  Dense(2, activation='softmax')  # Output layer for action probabilities
])

# Define the loss function (categorical cross-entropy)
loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

# Define the optimizer (Adam)
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

def choose_action(state):
  # Convert state to a tensor and add a batch dimension (for compatibility with model)
  state_tensor = tf.expand_dims(state, axis=0)

  # Predict action probabilities from the model
  action_probs = model(state_tensor)

  # Sample an action based on the probability distribution
  action = tf.random.categorical(action_probs, 1)[0]  # Sample one action
  return action.numpy()[0]  # Convert back to a NumPy value

def train(state, action, reward, next_state, done):
  # Convert state and next_state tensors to a batch dimension
  state_tensor = tf.expand_dims(state, axis=0)
  next_state_tensor = tf.expand_dims(next_state, axis=0)

  # Get one-hot encoded action (binary representation of chosen action)
  one_hot_action = tf.one_hot(action, env.action_space.n)

  # Calculate the target value (expected future reward)
  if done:
    target_value = tf.constant([reward], dtype=tf.float32) # Convert reward to a tensor with shape (1,)
  else:
    # Use Bellman equation to calculate target value (consider discounted future reward)
    q_values_next = model(next_state_tensor)
    target_value = reward + discount_factor * tf.reduce_max(q_values_next, axis=1)

  # Calculate the loss (difference between predicted Q-value and target value for the chosen action)
  with tf.GradientTape() as tape:
    q_values = model(state_tensor)
    q_action = tf.reduce_sum(q_values * one_hot_action, axis=1)
    loss = loss_fn(target_value, q_action)

  # Calculate gradients with respect to model parameters
  grads = tape.gradient(loss, model.trainable_variables)

  # Update model parameters using optimizer
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

# Training loop
for episode in range(episodes):
  state = env.reset()
  done = False
  score = 0

  while not done:
    action = choose_action(state)
    next_state, reward, done, info = env.step(action)
    score += reward

    train(state, action, reward, next_state, done)
    state = next_state

  print(f"Episode: {episode+1}, Score: {score}")

env.close()


